<a href="https://colab.research.google.com/github/Vimp17/gini_impurity/blob/main/%5Bhomework%5Dsimple_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-11-27 18:59:48--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-11-27 18:59:49--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  24.7MB/s    in 55s     

2024-11-27 19:00:44 (25.1 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [5]:
# method most_simmilar
similar_words = wv_embeddings.most_similar(word, topn=5)
similar_words

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [7]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        tokenizer: объект для токенизации текста
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    # Токенизируем текст вопроса
    tokens = tokenizer.tokenize(question.lower())

    # Список векторов для слов, которые присутствуют в embeddings
    word_vectors = []

    for word in tokens:
        if word in embeddings:
            word_vectors.append(embeddings[word])

    # Если ни одно слово из вопроса нет в embeddings, возвращаем нулевой вектор
    if not word_vectors:
        return np.zeros(dim)

    # Возвращаем среднее по векторным представлениям слов
    return np.mean(word_vectors, axis=0)


Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [8]:
sentence = 'I love neural networks'

sentence_vector = question_to_vec(sentence, wv_embeddings, tokenizer)

# Берем третью компоненту (индекс 2) и округляем до двух знаков
round(sentence_vector[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [13]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    return int(any(rank <= k for rank in dup_ranks))

In [15]:
def dcg_score(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов (начиная с 1)
    k: рассматриваем топ-k
    result: вернуть DCG@k (вещественное число)
    """
    dcg = 0.0
    for rank in dup_ranks:
        if rank <= k:
            # Добавляем вклад дубликата в DCG
            dcg += 1 / (np.log2(rank + 1))
    return dcg

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [11]:
import pandas as pd

In [17]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[0].index(ans) + 1 for ans in copy_answers if ans in candidates_ranking[0]]


# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [18]:
!unzip stackoverflow_similar_questions.zip

unzip:  cannot find or open stackoverflow_similar_questions.zip, stackoverflow_similar_questions.zip.zip or stackoverflow_similar_questions.zip.ZIP.


Считайте данные.

In [19]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            data.append(parts)
    return data


Нам понадобиться только файл validation.

In [22]:
validation_data = read_corpus('/content/validation.tsv')

Кол-во строк

In [23]:
len(validation_data)

1665

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [32]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
    Ранжирует кандидатов по сходству с вопросом.
    """
    # Преобразуем вопрос в векторное представление
    question_vec = question_to_vec(question, embeddings, tokenizer, dim)
    if question_vec.ndim == 1:
        question_vec = question_vec.reshape(1, -1)  # Преобразуем в 2D, если одномерный

    # Список для хранения кандидатов и их сходства
    candidates_with_scores = []

    for idx, candidate in enumerate(candidates):
        # Преобразуем кандидата в векторное представление
        candidate_vec = question_to_vec(candidate, embeddings, tokenizer, dim)
        if candidate_vec.ndim == 1:
            candidate_vec = candidate_vec.reshape(1, -1)  # Преобразуем в 2D, если одномерный

        # Вычисляем косинусное сходство
        similarity = cosine_similarity(question_vec, candidate_vec)

        # Сохраняем (исходный индекс, текст кандидата, сходство)
        candidates_with_scores.append((idx, candidate, similarity))

    # Сортируем кандидатов по сходству (по убыванию)
    candidates_with_scores.sort(key=lambda x: x[2], reverse=True)

    # Формируем результат в виде пар (исходный индекс, текст кандидата)
    ranked_candidates = [(idx, candidate) for idx, candidate, _ in candidates_with_scores]

    return ranked_candidates


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [30]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [33]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [34]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

SyntaxError: invalid syntax (<ipython-input-34-858aee104232>, line 5)

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

021

In [35]:
from tqdm.notebook import tqdm

In [38]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    rank_indices = [r[0] for r in ranks]
    if 0 in rank_indices:
        wv_ranking.append(rank_indices.index(0) + 1)
    else:
        wv_ranking.append(-1)  # Добавить -1, если индекс 0 отсутствует


  0%|          | 0/1665 [00:00<?, ?it/s]

In [39]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 415.000 | Hits@   1: 1.000
DCG@   5: 502.495 | Hits@   5: 1.000
DCG@  10: 524.731 | Hits@  10: 1.000
DCG@ 100: 570.256 | Hits@ 100: 1.000
DCG@ 500: 583.425 | Hits@ 500: 1.000
DCG@1000: 586.335 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [40]:
train_data = read_corpus('/content/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
'''your code'''

In [42]:
from gensim.models import Word2Vec
min_count = 5       # Игнорировать слова, которые встречаются меньше 5 раз
window = 5          # Слишком большое окно может снижать точность
size = 200          # Размер вектора эмбеддинга

# Обучение модели Word2Vec
embeddings_trained = Word2Vec(train_data,  # данные для обучения
                               vector_size=size,  # размер вектора
                               min_count=min_count,  # минимальная частота слов
                               window=window,  # размер окна
                               sg=0)  # sg=0 для CBOW, sg=1 для Skip-Gram

# Извлечение обученных эмбеддингов
embeddings = embeddings_trained.wv

In [45]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(train_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    rank_indices = [r[0] for r in ranks]
    if 0 in rank_indices:
        wv_ranking.append(rank_indices.index(0) + 1)
    else:
        wv_ranking.append(-1)  # Добавить -1, если индекс 0 отсутствует

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [46]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 889.000 | Hits@   1: 1.000
DCG@   5: 957.472 | Hits@   5: 1.000
DCG@  10: 957.472 | Hits@  10: 1.000
DCG@ 100: 957.472 | Hits@ 100: 1.000
DCG@ 500: 957.472 | Hits@ 500: 1.000
DCG@1000: 957.472 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Принцип токенизации и его влияние на качество:

Существует несколько подходов:
Токенизация по словам: Делит текст на отдельные слова. Этот метод удобен, но может иметь проблемы с редкими словами, омонимами и словами, состоящими из нескольких частей (например, с дефисами или апострофами).
Токенизация по субсловам (Subword Tokenization): Метод, который использует более мелкие единицы (например, символы или слоги). Это может быть полезно для работы с незнакомыми словами и помогает модели обрабатывать редкие и новые слова, что улучшает результат.
Токенизация по предложениям: Иногда лучше использовать токенизацию на уровне предложений, чтобы сохранить контекст. Это помогает, если задача требует более сложного контекста, чем отдельные слова.
Лучше всего в данном контексте работает токенизация, основанная на субсловах (например, Byte Pair Encoding — BPE или SentencePiece), так как она позволяет эффективно работать с редкими и неизвестными словами, а также помогает избежать проблем с морфологией.

Нормализация слов:

Нормализация, такая как приведение всех слов к нижнему регистру, удаление стоп-слов и исправление орфографических ошибок, может улучшить результаты модели. Это уменьшает разнообразие слов, с которыми работает модель, и помогает улучшить качество обучения.
Однако, в случае с субсловами или более сложной токенизацией нормализация может быть менее эффективной, так как многие современные эмбеддинги способны учитывать контекст без дополнительных манипуляций с текстом.
Какие эмбеддинги лучше справляются с задачей и почему?

Предобученные эмбеддинги: Эмбеддинги, такие как Word2Vec, GloVe и FastText, могут быть хорошими для задач, связанных с анализом текстов, так как они обладают многими полезными свойствами, такими как способность улавливать семантические зависимости между словами. Однако они могут плохо работать на редких или неизвестных словах.
Более современные модели: Модели типа BERT, RoBERTa, T5 и других трансформеров, обученных на большом количестве текстов, демонстрируют лучшие результаты для задач, где контекст важен. Эти модели учитывают не только отдельные слова, но и их контекст в предложении, что дает возможность улучшить точность.
В задаче обработки вопросов и нахождения похожих вопросов трансформеры, такие как BERT, могут продемонстрировать значительно более высокое качество, так как они обучаются на огромных корпусах текста и могут понимать тонкости контекста, в отличие от более простых эмбеддингов.
Почему качество решения задачи оказалось плохим?

Недостаточное количество данных: Модели, такие как Word2Vec или GloVe, требуют большого объема данных для хорошей генерализации. Недостаток данных может привести к плохим результатам.
Простота модели: Использование более простых эмбеддингов и классических моделей машинного обучения без использования современных нейронных сетей (например, трансформеров) может привести к недостаточному качеству.
Проблемы с токенизацией: Если токенизация не была настроена должным образом, это также может негативно сказаться на производительности модели. Например, проблемы с разделением синонимов или контекстных различий могут привести к некорректной оценке схожести вопросов.
Параметры модели: Выбор гиперпараметров, таких как размер окна для Word2Vec, может сильно влиять на качество. Слишком маленькое или большое окно может привести к потере важного контекста или избыточной информации.
Предложение подхода к решению задачи:

Использование трансформеров (например, BERT или T5): Эти модели обучены на огромных текстовых корпусах и способны эффективно работать с контекстом. Вместо простой модели Word2Vec стоит использовать трансформеры, которые могут учитывать как значение отдельных слов, так и их контекст.
Финетюнинг предобученной модели: Для задач, подобных этой, лучше использовать предобученную модель, например, BERT или его вариации, и затем дообучить её на специфических данных (например, на обучающих данных с вопросами и их похожими парами). Это улучшит качество, так как модель уже знает общие паттерны языка, а дообучение на специфической задаче сделает её более адаптированной.
Использование более сложных архитектур: Например, можно использовать Siamese Networks или Triplet Networks для задач поиска схожих вопросов. Эти архитектуры специально предназначены для задачи поиска похожих объектов, и они значительно улучшают качество.
Кросс-валидация и гиперпараметрическая оптимизация: Для дальнейшего улучшения качества можно использовать кросс-валидацию, чтобы избежать переобучения и подобрать оптимальные гиперпараметры.



```
# This is formatted as code
```

